In [2]:
# NOTE: PLEASE MAKE SURE YOU ARE RUNNING THIS IN A PYTHON3 ENVIRONMENT

import tensorflow as tf
print(tf.__version__)

# This is needed for the iterator over the data
# But not necessary if you have TF 2.0 installed
#!pip install tensorflow==2.0.0-beta0


# tf.enable_eager_execution()

# !pip install -q tensorflow-datasets

2.0.0-dev20191002


In [3]:
import tensorflow_datasets as tfds
imdb, info = tfds.load("imdb_reviews", with_info=True, as_supervised=True)


W1222 22:29:13.386377 16988 deprecation.py:323] From C:\Users\user\Anaconda3\lib\site-packages\tensorflow_datasets\core\file_format_adapter.py:209: tf_record_iterator (from tensorflow.python.lib.io.tf_record) is deprecated and will be removed in a future version.
Instructions for updating:
Use eager execution and: 
`tf.data.TFRecordDataset(path)`


Dataset imdb_reviews downloaded and prepared to C:\Users\user\tensorflow_datasets\imdb_reviews\plain_text\0.1.0. Subsequent calls will reuse this data.


W1222 22:30:01.886827 16988 dataset_builder.py:439] Warning: Setting shuffle_files=True because split=TRAIN and shuffle_files=None. This behavior will be deprecated on 2019-08-06, at which point shuffle_files=False will be the default for all splits.


In [4]:
import numpy as np

train_data, test_data = imdb['train'], imdb['test']

training_sentences = []
training_labels = []

testing_sentences = []
testing_labels = []

# str(s.tonumpy()) is needed in Python3 instead of just s.numpy()
for s,l in train_data:
  training_sentences.append(str(s.numpy()))
  training_labels.append(l.numpy())
  
for s,l in test_data:
  testing_sentences.append(str(s.numpy()))
  testing_labels.append(l.numpy())
  
training_labels_final = np.array(training_labels)
testing_labels_final = np.array(testing_labels)


In [5]:
vocab_size = 10000
embedding_dim = 16
max_length = 120
trunc_type='post'
oov_tok = "<OOV>"


from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

tokenizer = Tokenizer(num_words = vocab_size, oov_token=oov_tok)
tokenizer.fit_on_texts(training_sentences)
word_index = tokenizer.word_index
sequences = tokenizer.texts_to_sequences(training_sentences)
padded = pad_sequences(sequences,maxlen=max_length, truncating=trunc_type)

testing_sequences = tokenizer.texts_to_sequences(testing_sentences)
testing_padded = pad_sequences(testing_sequences,maxlen=max_length)



In [6]:
reverse_word_index = dict([(value, key) for (key, value) in word_index.items()])

def decode_review(text):
    return ' '.join([reverse_word_index.get(i, '?') for i in text])

print(decode_review(padded[1]))
print(training_sentences[1])

b take your basic frankenstein flick <OOV> some <OOV> but not the good parts and you have doctor <OOV> certainly this was obviously inspired by aforementioned films but it never <OOV> as anything special on its own br br a scientist accidentally kills his wife so the whole movie takes place over the course of one night as he attempts to revive his wife to revive his wife he decides to chop off body parts from some women that have become stranded and <OOV> decide to stay the night at his place br br i can't really say the acting is bad nor is the directing everything here is just way too standard what little attempts there are at humor
b"Take your basic Frankenstein flick, inject some Reanimator (but not the good parts), and you have Doctor Hackenstein. Certainly, this was obviously inspired by aforementioned films but it never materializes as anything special on its own.<br /><br />A scientist accidentally kills his wife, so the whole movie takes place over the course of one night as h

In [7]:
model = tf.keras.Sequential([
    tf.keras.layers.Embedding(vocab_size, embedding_dim, input_length=max_length),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(6, activation='relu'),
    tf.keras.layers.Dense(1, activation='sigmoid')
])
model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])
model.summary()


Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, 120, 16)           160000    
_________________________________________________________________
flatten (Flatten)            (None, 1920)              0         
_________________________________________________________________
dense (Dense)                (None, 6)                 11526     
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 7         
Total params: 171,533
Trainable params: 171,533
Non-trainable params: 0
_________________________________________________________________


In [8]:
num_epochs = 10
model.fit(padded, training_labels_final, epochs=num_epochs, validation_data=(testing_padded, testing_labels_final))

Train on 25000 samples, validate on 25000 samples
Epoch 1/10
25000/25000 [==============================] - ETA: 6:16 - loss: 0.6946 - accuracy: 0.53 - ETA: 14s - loss: 0.6920 - accuracy: 0.5190 - ETA: 7s - loss: 0.6932 - accuracy: 0.503 - ETA: 5s - loss: 0.6924 - accuracy: 0.51 - ETA: 4s - loss: 0.6916 - accuracy: 0.52 - ETA: 3s - loss: 0.6907 - accuracy: 0.52 - ETA: 2s - loss: 0.6886 - accuracy: 0.53 - ETA: 2s - loss: 0.6846 - accuracy: 0.55 - ETA: 2s - loss: 0.6782 - accuracy: 0.57 - ETA: 1s - loss: 0.6701 - accuracy: 0.58 - ETA: 1s - loss: 0.6616 - accuracy: 0.60 - ETA: 1s - loss: 0.6501 - accuracy: 0.61 - ETA: 1s - loss: 0.6375 - accuracy: 0.63 - ETA: 1s - loss: 0.6232 - accuracy: 0.64 - ETA: 1s - loss: 0.6093 - accuracy: 0.65 - ETA: 1s - loss: 0.5967 - accuracy: 0.66 - ETA: 0s - loss: 0.5849 - accuracy: 0.67 - ETA: 0s - loss: 0.5729 - accuracy: 0.68 - ETA: 0s - loss: 0.5645 - accuracy: 0.69 - ETA: 0s - loss: 0.5567 - accuracy: 0.69 - ETA: 0s - loss: 0.5479 - accuracy: 0.70 - ETA:

25000/25000 [==============================] - ETA: 1s - loss: 0.0017 - accuracy: 1.00 - ETA: 1s - loss: 0.0049 - accuracy: 0.99 - ETA: 1s - loss: 0.0040 - accuracy: 0.99 - ETA: 1s - loss: 0.0036 - accuracy: 0.99 - ETA: 1s - loss: 0.0033 - accuracy: 0.99 - ETA: 1s - loss: 0.0031 - accuracy: 0.99 - ETA: 1s - loss: 0.0030 - accuracy: 0.99 - ETA: 1s - loss: 0.0037 - accuracy: 0.99 - ETA: 1s - loss: 0.0035 - accuracy: 0.99 - ETA: 1s - loss: 0.0034 - accuracy: 0.99 - ETA: 1s - loss: 0.0033 - accuracy: 0.99 - ETA: 0s - loss: 0.0036 - accuracy: 0.99 - ETA: 0s - loss: 0.0041 - accuracy: 0.99 - ETA: 0s - loss: 0.0043 - accuracy: 0.99 - ETA: 0s - loss: 0.0047 - accuracy: 0.99 - ETA: 0s - loss: 0.0045 - accuracy: 0.99 - ETA: 0s - loss: 0.0047 - accuracy: 0.99 - ETA: 0s - loss: 0.0046 - accuracy: 0.99 - ETA: 0s - loss: 0.0045 - accuracy: 0.99 - ETA: 0s - loss: 0.0044 - accuracy: 0.99 - ETA: 0s - loss: 0.0045 - accuracy: 0.99 - ETA: 0s - loss: 0.0045 - accuracy: 0.99 - ETA: 0s - loss: 0.0048 - accu

In [9]:
e = model.layers[0]
weights = e.get_weights()[0]
print(weights.shape) # shape: (vocab_size, embedding_dim)

(10000, 16)


In [10]:
import io

out_v = io.open('vecs.tsv', 'w', encoding='utf-8')
out_m = io.open('meta.tsv', 'w', encoding='utf-8')
for word_num in range(1, vocab_size):
  word = reverse_word_index[word_num]
  embeddings = weights[word_num]
  out_m.write(word + "\n")
  out_v.write('\t'.join([str(x) for x in embeddings]) + "\n")
out_v.close()
out_m.close()

In [11]:


try:
  from google.colab import files
except ImportError:
  pass
else:
  files.download('vecs.tsv')
  files.download('meta.tsv')

In [12]:
sentence = "I really think this is amazing. honest."
sequence = tokenizer.texts_to_sequences(sentence)
print(sequence)

[[11], [], [1430], [968], [4], [1537], [1537], [4740], [], [790], [2016], [11], [2919], [2190], [], [790], [2016], [11], [579], [], [11], [579], [], [4], [1783], [4], [4508], [11], [2919], [1278], [], [], [2016], [1005], [2919], [968], [579], [790], []]
